In [2]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
from scipy.io import wavfile
import tensorflow as tf
import tensorflow.keras.backend as K
import itertools
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Load data
x_all = np.load('./data/x_all.npy') # RF
y_all = np.load('./data/y_all.npy') # QC Label

In [5]:
len(np.where(y_all==0)[0]),len(np.where(y_all==1)[0]),y_all.shape

(4614, 842, (5456,))

In [6]:
# split rate 0.2
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, shuffle=True, stratify=y_all)

x_train = x_train.reshape([x_train.shape[0], 600, 1]).astype(np.float32)
y_train = y_train.reshape([y_train.shape[0], 1]).astype(np.float32)

x_test = x_test.reshape([x_test.shape[0], 600, 1]).astype(np.float32)
y_test = y_test.reshape([y_test.shape[0], 1]).astype(np.float32)

In [ ]:
# balanced sampling
pos_ids = np.where(y_train==1)
x_train_pos = x_train[pos_ids[0],:,:]
y_train_pos = np.zeros([x_train_pos.shape[0],2])
y_train_pos[:,0] = 1

neg_ids = np.where(y_train==0)
x_train_neg = x_train[neg_ids[0],:,:]
y_train_neg = np.zeros([x_train_neg.shape[0],2])
y_train_neg[:,1] = 1

pos_dataset = tf.data.Dataset.from_tensor_slices((x_train_pos, y_train_pos)).repeat()
neg_dataset = tf.data.Dataset.from_tensor_slices((x_train_neg, y_train_neg)).repeat()
balanced_dataset = tf.data.experimental.sample_from_datasets([pos_dataset, neg_dataset],weights=[0.5, 0.5])

# reshape y_test
pos_ids = np.where(y_test==1)
x_test_pos = x_test[pos_ids[0],:,:]
y_test_pos = np.zeros([x_test_pos.shape[0],2])
y_test_pos[:,0] = 1

neg_ids = np.where(y_test==0)
x_test_neg = x_test[neg_ids[0],:,:]
y_test_neg = np.zeros([x_test_neg.shape[0],2])
y_test_neg[:,1] = 1

pos_dataset_test = tf.data.Dataset.from_tensor_slices((x_test_pos, y_test_pos)).repeat()
neg_dataset_test = tf.data.Dataset.from_tensor_slices((x_test_neg, y_test_neg)).repeat()
balanced_dataset_test = tf.data.experimental.sample_from_datasets([pos_dataset_test, neg_dataset_test],weights=[0.5, 0.5])

# reshape y_test
pos_val_ids = np.where(y_test==1)
neg_val_ids = np.where(y_test==0)
y_test_temp = np.zeros([y_test.shape[0], 2])
y_test_temp[pos_val_ids[0], 0] = 1
y_test_temp[neg_val_ids[0], 1] = 1
y_test = y_test_temp

# Capsule

In [8]:
import numpy as np
import h5py
import glob
import math
import os
import shutil
from scipy import signal
from scipy.signal import butter, lfilter
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dropout, Reshape 
from tensorflow.keras.layers import Bidirectional, concatenate, BatchNormalization, ZeroPadding1D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.callbacks import CSVLogger
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
from tensorflow.python.keras.layers import Layer, InputSpec
import tensorflow as tf
#tf.compat.v1.reset_default_graph()
from sklearn.metrics import accuracy_score

import numpy as np
import h5py
import glob
import math
import os
import shutil
from scipy import signal
from scipy.signal import butter, lfilter
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Dropout, Reshape 
from tensorflow.keras.layers import Bidirectional, concatenate, BatchNormalization, ZeroPadding1D, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.initializers import VarianceScaling
from tensorflow.keras.callbacks import CSVLogger
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
from tensorflow.python.keras.layers import Layer, InputSpec
import tensorflow as tf
tf.compat.v1.reset_default_graph()
from sklearn.metrics import accuracy_score

from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from keras.layers import Activation, Add, Bidirectional, Conv1D, Dense, Dropout, Embedding, Flatten, Reshape, multiply
from keras.layers import concatenate, GRU, Input, LSTM, MaxPooling1D
from keras.layers import GlobalAveragePooling1D,  GlobalMaxPooling1D, SpatialDropout1D
from keras.models import Model
from keras.preprocessing import text, sequence
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.models import Model



In [9]:
########## 400k model
from capsulelayers import *

def CapsNet(input_shape, n_class, routings):
    """
    A Capsule Network on MNIST.
    :param input_shape: data shape, 3d, [width, height, channels]
    :param n_class: number of classes
    :param routings: number of routing iterations
    :return: Two Keras Models, the first one used for training, and the second one for evaluation.
            `eval_model` can also be used for training.
    """
    
    ze = 16
    x = Input(shape=input_shape)

    # Layer 1: Just a conventional Conv2D layer
    #conv1 = Conv1D(filters=4, kernel_size=3, strides=1, padding='same', activation='relu', name='conv1')(x)
    #conv1 = Dropout(0.5)(conv1)
    #conv2 = layers.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='rrelu', name='conv2')(conv1)
    #conv3 = layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='rrelu', name='conv3')(conv2)

    # Layer 2: Conv2D layer with `squash` activation, then reshape to [None, num_capsule, dim_capsule]
    primarycaps = PrimaryCap(x, dim_capsule=8, n_channels=8, kernel_size=3, strides=1, padding='same')
    primarycaps = Dropout(0.8)(primarycaps)

    # Layer 3: Capsule layer. Routing algorithm works here.
    digitcaps = CapsuleLayer(num_capsule=n_class, dim_capsule=ze, routings=routings,
                             name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='capsnet')(digitcaps)



    # Models for training and evaluation (prediction)
    model = Model([x], [out_caps])


    return model


In [10]:
model = CapsNet(input_shape=(600,1), n_class=2, routings=3)
model.summary()

Instructions for updating:
Use fn_output_signature instead
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 600, 1)]          0         
                                                                 
 primarycap_conv2d (Conv1D)  (None, 600, 64)           256       
                                                                 
 primarycap_reshape (Reshape  (None, 4800, 8)          0         
 )                                                               
                                                                 
 primarycap_squash (Lambda)  (None, 4800, 8)           0         
                                                                 
 dropout (Dropout)           (None, 4800, 8)           0         
                                                                 
 digitcaps (CapsuleLayer)    (None, 2, 16)             1228800   
  

In [11]:
def margin_loss(y_true, y_pred):

    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
    0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))

    return K.mean(K.sum(L, 1))

In [12]:
import keras
import tensorflow
#costom loss for calculating aleatoric uncertainty
K.clear_session()


model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Recall()])

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                cooldown= 0,
                                patience= 20,
                                min_lr=0.5e-8)

early_stopping_monitor = EarlyStopping(monitor= 'val_accuracy', patience = 50)

checkpoint = ModelCheckpoint(filepath='best_model_CAPS_ALLX.h5',
                             monitor='val_accuracy',
                             mode = 'max',
                             verbose=1,
                             save_best_only=True)


callbacks = [lr_reducer, early_stopping_monitor, checkpoint]

history1 = model.fit(balanced_dataset.repeat().shuffle(1000).batch(8), steps_per_epoch=200, epochs=2000, validation_data=balanced_dataset_test.shuffle(100).batch(8),validation_steps=200, callbacks = callbacks)


Epoch 1/2000


2024-04-01 14:48:41.353713: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2024-04-01 14:48:42.029704: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2024-04-01 14:48:42.093401: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


197/200 [============================>.] - ETA: 0s - loss: 0.5910 - accuracy: 0.5044 - recall: 0.0076
Epoch 1: val_accuracy improved from -inf to 0.47188, saving model to best_model_CAPS_ALLX.h5
200/200 [==============================] - 8s 19ms/step - loss: 0.5912 - accuracy: 0.5031 - recall: 0.0075 - val_loss: 0.5275 - val_accuracy: 0.4719 - val_recall: 0.0150 - lr: 1.0000e-04
Epoch 2/2000
197/200 [============================>.] - ETA: 0s - loss: 0.4698 - accuracy: 0.7037 - recall: 0.0209
Epoch 2: val_accuracy improved from 0.47188 to 0.87437, saving model to best_model_CAPS_ALLX.h5
200/200 [==============================] - 3s 16ms/step - loss: 0.4673 - accuracy: 0.7063 - recall: 0.0237 - val_loss: 0.3688 - val_accuracy: 0.8744 - val_recall: 0.0319 - lr: 1.0000e-04
Epoch 3/2000
199/200 [============================>.] - ETA: 0s - loss: 0.3704 - accuracy: 0.8612 - recall: 0.0270
Epoch 3: val_accuracy improved from 0.87437 to 0.88375, saving model to best_model_CAPS_ALLX.h5
200/200 [

200/200 [==============================] - 3s 16ms/step - loss: 0.2316 - accuracy: 0.9287 - recall: 0.0881 - val_loss: 0.2924 - val_accuracy: 0.8856 - val_recall: 0.0875 - lr: 1.0000e-04
Epoch 24/2000
196/200 [============================>.] - ETA: 0s - loss: 0.2263 - accuracy: 0.9298 - recall: 0.0989
Epoch 24: val_accuracy did not improve from 0.89812
200/200 [==============================] - 3s 16ms/step - loss: 0.2260 - accuracy: 0.9312 - recall: 0.1000 - val_loss: 0.3010 - val_accuracy: 0.8819 - val_recall: 0.0875 - lr: 1.0000e-04
Epoch 25/2000
195/200 [============================>.] - ETA: 0s - loss: 0.2268 - accuracy: 0.9365 - recall: 0.0782
Epoch 25: val_accuracy did not improve from 0.89812
200/200 [==============================] - 3s 16ms/step - loss: 0.2257 - accuracy: 0.9369 - recall: 0.0794 - val_loss: 0.2935 - val_accuracy: 0.8888 - val_recall: 0.0900 - lr: 1.0000e-04
Epoch 26/2000
196/200 [============================>.] - ETA: 0s - loss: 0.2296 - accuracy: 0.9324 - re

Epoch 47/2000
198/200 [============================>.] - ETA: 0s - loss: 0.2095 - accuracy: 0.9463 - recall: 0.0909
Epoch 47: val_accuracy did not improve from 0.89812
200/200 [==============================] - 3s 16ms/step - loss: 0.2090 - accuracy: 0.9463 - recall: 0.0906 - val_loss: 0.2894 - val_accuracy: 0.8869 - val_recall: 0.0975 - lr: 1.0000e-04
Epoch 48/2000
197/200 [============================>.] - ETA: 0s - loss: 0.2046 - accuracy: 0.9499 - recall: 0.0888
Epoch 48: val_accuracy did not improve from 0.89812
200/200 [==============================] - 3s 16ms/step - loss: 0.2049 - accuracy: 0.9494 - recall: 0.0906 - val_loss: 0.2963 - val_accuracy: 0.8831 - val_recall: 0.0950 - lr: 1.0000e-04
Epoch 49/2000
199/200 [============================>.] - ETA: 0s - loss: 0.2103 - accuracy: 0.9466 - recall: 0.0861
Epoch 49: val_accuracy did not improve from 0.89812
200/200 [==============================] - 3s 16ms/step - loss: 0.2099 - accuracy: 0.9469 - recall: 0.0875 - val_loss: 0.2

Epoch 70/2000
195/200 [============================>.] - ETA: 0s - loss: 0.2054 - accuracy: 0.9545 - recall: 0.0974
Epoch 70: val_accuracy did not improve from 0.89938
200/200 [==============================] - 3s 16ms/step - loss: 0.2050 - accuracy: 0.9544 - recall: 0.0962 - val_loss: 0.2799 - val_accuracy: 0.8875 - val_recall: 0.0969 - lr: 1.0000e-04
Epoch 71/2000
197/200 [============================>.] - ETA: 0s - loss: 0.1933 - accuracy: 0.9454 - recall: 0.0831
Epoch 71: val_accuracy did not improve from 0.89938
200/200 [==============================] - 3s 17ms/step - loss: 0.1933 - accuracy: 0.9463 - recall: 0.0844 - val_loss: 0.2894 - val_accuracy: 0.8881 - val_recall: 0.0975 - lr: 1.0000e-04
Epoch 72/2000
200/200 [==============================] - ETA: 0s - loss: 0.1992 - accuracy: 0.9481 - recall: 0.0925
Epoch 72: val_accuracy did not improve from 0.89938
200/200 [==============================] - 3s 16ms/step - loss: 0.1992 - accuracy: 0.9481 - recall: 0.0925 - val_loss: 0.2

In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, r2_score

model.load_weights('best_model_CAPS_ALLX.h5')
out = model.predict(x_test)

thre = 0.5
#outB = np.zeros_like(out)
#outB[out>=thre] = 1
outB = np.argmax(out,axis=-1)
Y_test1 = np.argmax(y_test,axis=-1)
accuracy_score(Y_test1,outB),precision_score(Y_test1,outB),recall_score(Y_test1,outB),f1_score(Y_test1,outB)

35/35 [==============================] - 1s 11ms/step


(0.88003663003663, 0.9864864864864865, 0.8699891657638137, 0.9245826137017847)